In [ ]:
# -*- coding: utf-8 -*-
"""Lane_Detection_Colab_Final.ipynb

Original file is located at
    https://colab.research.google.com/drive/your_notebook_id

# Advanced Lane Detection with Direction
"""

# Install required packages
!pip install opencv-python moviepy ultralytics gitpython
!pip install huggingface_hub
!pip install --upgrade ultralytics

# Import required libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
from PIL import ImageFont, ImageDraw, Image
from google.colab import files, drive
import os
from ultralytics import YOLO
import git
from huggingface_hub import hf_hub_download
import time
import shutil

"""## 1. 코랩 및 yolo/캘리브레이션 데이터"""

# Global variables
CAMERA_MTX = None
CAMERA_DIST = None
model = None  # Global YOLO model variable
PREV_METRICS = None
SMOOTHING_FACTOR = 0.3  # Adjust this value to control smoothing (0-1)
FRAME_TIME = time.time()
SPEED_MPH = 0

def setup_directories():
    """Setup necessary directories in Google Drive."""
    # Mount Google Drive
    drive.mount('/content/drive')

    # Define base directory in Google Drive
    base_dir = '/content/drive/MyDrive/lane_detection'

    # Create necessary directories
    directories = ['input', 'output', 'models', 'calibration']
    for dir_name in directories:
        dir_path = os.path.join(base_dir, dir_name)
        os.makedirs(dir_path, exist_ok=True)
        print(f"Created directory: {dir_path}")

    return base_dir

def download_yolo_model(base_dir):
    """Download YOLO model from Hugging Face."""
    global model
    model_path = os.path.join(base_dir, 'models', 'yolo11n-seg.pt')

    if not os.path.exists(model_path):
        print("Downloading YOLO model...")
        try:
            # Direct download command for YOLO model
            !wget -P {os.path.dirname(model_path)} https://huggingface.co/Ultralytics/YOLO11/resolve/main/yolo11n-seg.pt
            print("YOLO model downloaded successfully!")
        except Exception as e:
            print(f"Error downloading YOLO model: {str(e)}")
            return None

    # Initialize YOLO model
    try:
        model = YOLO(model_path)
        print("YOLO model loaded successfully!")
    except Exception as e:
        print(f"Error loading YOLO model: {str(e)}")
        return None

    return model_path

def clone_calibration_data(base_dir):
    """Clone calibration data from Hugging Face repository."""
    calibration_dir = os.path.join(base_dir, 'calibration')

    if not os.path.exists(os.path.join(calibration_dir, 'camera_cal')):
        print("Cloning calibration data...")
        repo_url = "https://huggingface.co/sdgsdggds/camera"
        git.Repo.clone_from(repo_url, calibration_dir)
        print("Calibration data cloned successfully!")

    return os.path.join(calibration_dir, 'camera_cal')

def initialize_system():
    """Initialize the entire system."""
    print("Setting up the system...")

    # Setup directories
    base_dir = setup_directories()

    # Download YOLO model
    model_path = download_yolo_model(base_dir)

    # Clone calibration data
    calibration_dir = clone_calibration_data(base_dir)

    return base_dir, calibration_dir


"""## 2. Camera Calibration"""

def initialize_calibration(calibration_dir):
    """Initialize camera calibration using chessboard images."""
    global CAMERA_MTX, CAMERA_DIST
    nx = 9  # number of inside corners in x
    ny = 6  # number of inside corners in y
    objpoints = []
    imgpoints = []

    # Prepare object points
    objp = np.zeros((ny*nx,3), np.float32)
    objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)

    # Calibration images should be in a folder named 'camera_cal' in your Google Drive
    cal_images = [f for f in os.listdir(calibration_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

    if not cal_images:
        print("No calibration images found in 'camera_cal' folder!")
        return False

    for fname in cal_images:
        img = cv2.imread(os.path.join(calibration_dir, fname))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

        if ret:
            imgpoints.append(corners)
            objpoints.append(objp)

    if len(objpoints) == 0:
        print("No valid calibration images found!")
        return False

    # Calibrate camera
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    CAMERA_MTX = mtx
    CAMERA_DIST = dist
    print("Camera calibration completed successfully!")
    return True

"""## 3. Image Processing Functions"""

def warp(img, mtx, dist):
    """Apply perspective transform to input image."""
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    img_size = (img.shape[1], img.shape[0])
    offset = 300

    src = np.float32([
        (190, 720),   # Bottom left
        (596, 447),   # Top left
        (685, 447),   # Top right
        (1125, 720)   # Bottom right
    ])

    dst = np.float32([
        [offset, img_size[1]],             # Bottom left
        [offset, 0],                       # Top left
        [img_size[0]-offset, 0],           # Top right
        [img_size[0]-offset, img_size[1]]  # Bottom right
    ])

    M = cv2.getPerspectiveTransform(src, dst)
    M_inv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(undist, M, img_size)

    return warped, M_inv

def binary_thresholded(img):
    """Create binary thresholded image using color and gradient thresholds."""
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Sobel x
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    abs_sobelx = np.absolute(sobelx)
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    sx_binary = np.zeros_like(scaled_sobel)
    sx_binary[(scaled_sobel >= 30) & (scaled_sobel <= 255)] = 1

    # Threshold for white pixels
    white_binary = np.zeros_like(gray)
    white_binary[(gray > 200) & (gray <= 255)] = 1

    # Threshold for yellow lines using HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    H = hls[:,:,0]
    S = hls[:,:,2]

    sat_binary = np.zeros_like(S)
    sat_binary[(S > 90) & (S <= 255)] = 1

    hue_binary = np.zeros_like(H)
    hue_binary[(H > 10) & (H <= 25)] = 1

    # Combine thresholds
    binary_1 = cv2.bitwise_or(sx_binary, white_binary)
    binary_2 = cv2.bitwise_or(sat_binary, hue_binary)
    binary = cv2.bitwise_or(binary_1, binary_2)

    return binary

def combine_images(main_image, process_images, labels=None):
    """Combine main image with processing steps displayed vertically."""
    # Get dimensions
    h, w = main_image.shape[:2]

    # Calculate dimensions for processing images
    proc_height = h // len(process_images)  # Height for each processing image
    proc_width = w // 2  # Width for processing images (half of main image)

    # Resize main image to take up left half
    main_resized = cv2.resize(main_image, (proc_width*2, h))

    # Process and resize the processing images
    processed = []
    for img in process_images:
        # Convert if grayscale
        if len(img.shape) == 2:
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        # Resize to fit the right half
        img_resized = cv2.resize(img, (proc_width, proc_height))
        processed.append(img_resized)

    # Stack processing images vertically
    right_side = np.vstack(processed)

    # Combine main image with processing images
    combined = np.hstack((main_resized, right_side))

    # Add labels if provided
    if labels:
        for i, label in enumerate(labels):
            x = proc_width * 2 + 10  # Position labels on right side
            y = (i * proc_height) + 30  # Vertical position for each label
            cv2.putText(combined, label, (x, y),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    return combined

"""## 4. Lane Detection Functions"""

def find_lane_pixels(binary_warped):
    """Find lane pixels in the binary warped image."""
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)

    midpoint = int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    nwindows = 9
    margin = 100
    minpix = 50

    window_height = int(binary_warped.shape[0]//nwindows)

    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    leftx_current = leftx_base
    rightx_current = rightx_base

    left_lane_inds = []
    right_lane_inds = []

    out_img = np.dstack((binary_warped, binary_warped, binary_warped)) * 255

    for window in range(nwindows):
        win_y_low = binary_warped.shape[0] - (window + 1) * window_height
        win_y_high = binary_warped.shape[0] - window * window_height

        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin

        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                         (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                          (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]

        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        if len(good_left_inds) > minpix:
            leftx_current = int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = int(np.mean(nonzerox[good_right_inds]))

        cv2.rectangle(out_img, (win_xleft_low, win_y_low), (win_xleft_high, win_y_high), (0, 255, 0), 2)
        cv2.rectangle(out_img, (win_xright_low, win_y_low), (win_xright_high, win_y_high), (0, 255, 0), 2)

    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return out_img, leftx, lefty, rightx, righty

def determine_curve_direction(left_fit, right_fit):
    """Determine the direction of the curve based on polynomial coefficients."""
    if left_fit is None or right_fit is None:
        return "Unknown"

    avg_curve = (left_fit[0] + right_fit[0]) / 2

    if abs(avg_curve) < 5e-4:  # 임계값을 1e-3에서 5e-4로 감소
        return "↑↑"
    elif avg_curve > 0:
        return "→→"
    else:
        return "←←"

def fit_polynomial(binary_warped):
    """Fit polynomial to binary image with lane lines."""
    _, leftx, lefty, rightx, righty = find_lane_pixels(binary_warped)

    if len(leftx) == 0 or len(rightx) == 0:
        return None, None

    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    return left_fit, right_fit

def calculate_tilt(left_fit, right_fit, img_shape):
    """Calculate the tilt value and direction based on lane line polynomials."""
    # Calculate position of lines at the bottom of the image
    y_eval = img_shape[0]
    left_x = left_fit[0]*y_eval**2 + left_fit[1]*y_eval + left_fit[2]
    right_x = right_fit[0]*y_eval**2 + right_fit[1]*y_eval + right_fit[2]

    # Calculate center position
    center_pos = (left_x + right_x) / 2
    image_center = img_shape[1] / 2

    # Calculate offset from center (positive is right tilt, negative is left tilt)
    offset = center_pos - image_center

    # Convert offset to a tilt value (scale factor can be adjusted)
    tilt_value = round(offset / (image_center) * 100, 1)  # Convert to percentage

    if abs(tilt_value) < 1.0:  # Threshold for considering it centered
        return 0, "center"
    elif tilt_value > 0:
        return tilt_value, "right"
    else:
        return abs(tilt_value), "left"

def smooth_metrics(new_metrics, prev_metrics):
    """Apply exponential smoothing to metrics."""
    if prev_metrics is None:
        return new_metrics

    smoothed = {}
    for key in new_metrics:
        if key in prev_metrics:
            smoothed[key] = (SMOOTHING_FACTOR * new_metrics[key] +
                           (1 - SMOOTHING_FACTOR) * prev_metrics[key])
        else:
            smoothed[key] = new_metrics[key]
    return smoothed

def estimate_speed(curve_radius, frame_time):
    """Estimate speed based on curve radius and frame time."""
    global SPEED_MPH

    # Get time difference
    current_time = time.time()
    dt = current_time - frame_time

    # Estimate speed based on curve radius (larger radius = higher possible speed)
    # This is a simplified model and would need calibration for accurate results
    max_speed = min(75, curve_radius / 30)  # mph, capped at 75 mph
    target_speed = max_speed * 0.8  # Assume driving at 80% of max safe speed

    # Smooth speed changes
    SPEED_MPH = SPEED_MPH * 0.9 + target_speed * 0.1

    return SPEED_MPH, current_time

def check_lane_departure(position_offset, lane_width):
    """Check if vehicle is departing from lane."""
    # Warning thresholds (as percentage of lane width)
    warning_threshold = lane_width * 0.3
    critical_threshold = lane_width * 0.4

    if abs(position_offset) > critical_threshold:
        return "CRITICAL", (0, 0, 255)  # Red for critical
    elif abs(position_offset) > warning_threshold:
        return "WARNING", (0, 255, 255)  # Yellow for warning
    return "NORMAL", (0, 255, 0)  # Green for normal

def calculate_metrics(left_fit, right_fit, binary_warped, leftx, lefty, rightx, righty):
    """Calculate various lane metrics."""
    global PREV_METRICS, FRAME_TIME

    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720  # meters per pixel in y dimension
    xm_per_pix = 3.7/700  # meters per pixel in x dimension

    # Calculate curve radius
    y_eval = binary_warped.shape[0]

    # Fit polynomials in world space
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)

    # Calculate curve radius
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) \
                    / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) \
                    / np.absolute(2*right_fit_cr[0])
    curve_radius = round((left_curverad + right_curverad) / 2, 1)

    # Calculate lane width
    left_x = left_fit[0]*y_eval**2 + left_fit[1]*y_eval + left_fit[2]
    right_x = right_fit[0]*y_eval**2 + right_fit[1]*y_eval + right_fit[2]
    lane_width = (right_x - left_x) * xm_per_pix
    lane_width = round(lane_width, 2)

    # Calculate vehicle position offset
    center_pos = (left_x + right_x) / 2
    image_center = binary_warped.shape[1] / 2
    offset = (center_pos - image_center) * xm_per_pix
    position_offset = round(offset, 2)

    # Calculate lane detection confidence score (0-100)
    # Based on number of detected pixels and lane width consistency
    expected_width = 3.7  # meters
    width_score = max(0, 100 - abs(lane_width - expected_width) * 50)
    pixel_score = min(100, (len(leftx) + len(rightx)) / 50)
    confidence_score = round((width_score + pixel_score) / 2, 1)

    # Calculate speed
    speed, new_frame_time = estimate_speed(curve_radius, FRAME_TIME)
    FRAME_TIME = new_frame_time

    metrics = {
        'curve_radius': curve_radius,
        'lane_width': lane_width,
        'position_offset': position_offset,
        'confidence': confidence_score,
        'speed': speed
    }

    # Apply smoothing
    smoothed_metrics = smooth_metrics(metrics, PREV_METRICS)
    PREV_METRICS = smoothed_metrics

    return smoothed_metrics

def draw_metrics(image, metrics):
    """Draw lane metrics on the image."""
    h, w = image.shape[:2]

    # Check lane departure status
    departure_status, status_color = check_lane_departure(
        metrics['position_offset'], metrics['lane_width'])

    # Define metrics positions (left side of image)
    start_x = 10
    start_y = 30
    line_height = 30

    # Metrics text with units
    metrics_text = [
        f"Curve Radius: {metrics['curve_radius']:.1f}m",
        f"Lane Width: {metrics['lane_width']:.2f}m",
        f"Position Offset: {metrics['position_offset']:+.2f}m",
        f"Speed: {metrics['speed']:.1f}mph",
        f"Confidence: {metrics['confidence']:.1f}%"
    ]

    # Draw background rectangle for better visibility
    padding = 10
    rect_height = len(metrics_text) * line_height + 2 * padding
    rect_width = 300
    cv2.rectangle(image, (start_x - padding, start_y - padding - 20),
                 (start_x + rect_width, start_y + rect_height),
                 (0, 0, 0), -1)
    cv2.rectangle(image, (start_x - padding, start_y - padding - 20),
                 (start_x + rect_width, start_y + rect_height),
                 (255, 255, 255), 1)

    # Draw metrics text
    for i, text in enumerate(metrics_text):
        y_pos = start_y + i * line_height
        cv2.putText(image, text, (start_x, y_pos),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Draw lane departure warning if not normal
    if departure_status != "NORMAL":
        warning_text = f"Lane Departure {departure_status}!"
        text_size = cv2.getTextSize(warning_text, cv2.FONT_HERSHEY_SIMPLEX, 1.0, 2)[0]
        warn_x = (w - text_size[0]) // 2
        cv2.putText(image, warning_text, (warn_x, h - 50),
                   cv2.FONT_HERSHEY_SIMPLEX, 1.0, status_color, 2)

    return image

def draw_tilt_indicator(image, tilt_value, direction):
    """Draw tilt indicator with purple line and value."""
    h, w = image.shape[:2]
    center_x = w // 2
    center_y = h // 2
    line_length = 50  # Length of the indicator line

    # Draw center vertical line
    cv2.line(image, (center_x, center_y - 60), (center_x, center_y + 60), (128, 128, 128), 1)

    if direction == "center":
        # Draw horizontal line in the center
        cv2.line(image, (center_x - line_length//2, center_y),
                (center_x + line_length//2, center_y), (255, 0, 255), 2)
        text = "0"
    else:
        if direction == "right":
            # Draw tilted line to the right
            end_x = center_x + line_length//2
            end_y = center_y - 20
            text = f"right: {tilt_value}"
        else:  # left
            # Draw tilted line to the left
            end_x = center_x - line_length//2
            end_y = center_y - 20
            text = f"left: {tilt_value}"

        cv2.line(image, (center_x, center_y), (end_x, end_y), (255, 0, 255), 2)

    # Draw text
    cv2.putText(image, text, (center_x - 40, center_y - 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    return image

"""## 5. Main Processing Functions"""

def process_image(image):
    """Process image to detect lane lines, objects, and determine direction."""
    global CAMERA_MTX, CAMERA_DIST, model

    if CAMERA_MTX is None or CAMERA_DIST is None:
        print("Camera calibration required!")
        return image

    if model is None:
        print("YOLO model not initialized!")
        return image

    # Make a copy of the original image
    result = image.copy()

    try:
        # YOLO object detection
        results = model(image)

        # Draw YOLO detections
        for r in results:
            boxes = r.boxes
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                cls = int(box.cls[0])
                conf = float(box.conf[0])
                name = model.names[cls]

                if conf > 0.5:
                    cv2.rectangle(result, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(result, f'{name} {conf:.2f}', (x1, y1 - 10),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    except Exception as e:
        print(f"Error in YOLO detection: {str(e)}")

    # Lane detection steps
    warped, M_inv = warp(result, CAMERA_MTX, CAMERA_DIST)
    binary = binary_thresholded(warped)

    # Get line pixels visualization
    out_img, leftx, lefty, rightx, righty = find_lane_pixels(binary)

    # Fit polynomials
    left_fit, right_fit = fit_polynomial(binary)

    if left_fit is None or right_fit is None:
        return result

    # Calculate metrics
    metrics = calculate_metrics(left_fit, right_fit, binary, leftx, lefty, rightx, righty)

    # Calculate tilt
    tilt_value, tilt_direction = calculate_tilt(left_fit, right_fit, image.shape)

    # Generate points for plotting
    ploty = np.linspace(0, image.shape[0]-1, image.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # Create lane overlay
    warp_zero = np.zeros_like(binary).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))
    newwarp = cv2.warpPerspective(color_warp, M_inv, (image.shape[1], image.shape[0]))

    # Combine lane detection with result image
    result = cv2.addWeighted(result, 1, newwarp, 0.3, 0)

    # Draw metrics
    result = draw_metrics(result, metrics)

    # Draw tilt indicator
    result = draw_tilt_indicator(result, tilt_value, tilt_direction)

    # Create visualization of binary and warped images
    binary_colored = cv2.cvtColor(binary * 255, cv2.COLOR_GRAY2BGR)
    warped_viz = warped.copy()

    # Draw detected lane lines on warped image
    for fitx, color in [(left_fitx, (255, 0, 0)), (right_fitx, (0, 0, 255))]:
        pts = np.array([np.transpose(np.vstack([fitx, ploty]))]).astype(np.int32)
        cv2.polylines(warped_viz, pts, False, color, thickness=2)

    # Create line pixels visualization
    line_pixels_viz = np.dstack((binary, binary, binary)) * 255
    # Plot line pixels
    line_pixels_viz[lefty, leftx] = [255, 0, 0]  # Red for left line
    line_pixels_viz[righty, rightx] = [0, 0, 255]  # Blue for right line
    # Draw search windows if available from find_lane_pixels
    if hasattr(out_img, 'shape'):
        window_img = cv2.addWeighted(line_pixels_viz, 1, out_img, 0.3, 0)
        line_pixels_viz = window_img

    # Combine all visualizations
    process_images = [binary_colored, warped_viz, color_warp, line_pixels_viz]
    labels = ['Binary Threshold', 'Warped Perspective', 'Lane Detection', 'Line Pixels']
    combined_result = combine_images(result, process_images, labels)

    return combined_result

def process_video(input_path, output_path):
    """Process video file for lane detection."""
    clip = VideoFileClip(input_path)

    def resize_frame(frame):
        # 화면비를 유지하면서 1280x768으로 리사이즈
        return cv2.resize(frame, (1280, 720), interpolation=cv2.INTER_LANCZOS4)

    def process_frame(frame):
        # 프레임 리사이즈 후 처리
        resized_frame = resize_frame(frame)
        return process_image(resized_frame)

    processed_clip = clip.fl_image(process_frame)
    processed_clip.write_videofile(output_path, audio=False)
    clip.close()

"""## 6. Main Execution"""

def main():
    """Main function to run the lane detection pipeline."""
    # Initialize system
    base_dir, calibration_dir = initialize_system()

    print("Starting camera calibration...")
    if not initialize_calibration(calibration_dir):
        print("Camera calibration failed!")
        return

    # Input/Output paths
    input_dir = os.path.join(base_dir, 'input')
    output_dir = os.path.join(base_dir, 'output')

    print("\nChecking for video files in the input directory...")
    print(f"Input directory: {input_dir}")

    # Process all video files in the input directory
    while True:
        video_files = [f for f in os.listdir(input_dir) if f.endswith(('.mp4', '.avi'))]
        if video_files:
            break
        print("Waiting for video files... Please upload to input directory.")
        time.sleep(5)

    total_videos = len(video_files)
    print(f"\nFound {total_videos} video file(s) to process.")

    for idx, video_file in enumerate(video_files, 1):
        print(f"\nProcessing video {idx}/{total_videos}: {video_file}")
        input_video = os.path.join(input_dir, video_file)
        output_video = os.path.join(output_dir, f"output_{video_file}")

        try:
            process_video(input_video, output_video)
            print(f"Successfully processed: {video_file}")
            print(f"Output saved to: {output_video}")
        except Exception as e:
            print(f"Error processing {video_file}: {str(e)}")
            continue

    print("\nAll video processing complete!")
    print("You can find the processed videos in the output directory.")

if __name__ == "__main__":
    main()


Setting up the system...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Created directory: /content/drive/MyDrive/lane_detection/input
Created directory: /content/drive/MyDrive/lane_detection/output
Created directory: /content/drive/MyDrive/lane_detection/models
Created directory: /content/drive/MyDrive/lane_detection/calibration
YOLO model loaded successfully!
Starting camera calibration...
Camera calibration completed successfully!

Checking for video files in the input directory...
Input directory: /content/drive/MyDrive/lane_detection/input

Found 1 video file(s) to process.

Processing video 1/1: 3.mp4

0: 384x640 2 cars, 1 truck, 14.0ms
Speed: 1.9ms preprocess, 14.0ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
Moviepy - Building video /content/drive/MyDrive/lane_detection/output/output_3.mp4.
Moviepy - Writing video /content/drive/MyDrive/lane_detection/output/output_3.mp4



t:   0%|          | 0/1800 [00:00<?, ?it/s, now=None]


0: 384x640 2 cars, 1 truck, 10.2ms
Speed: 2.8ms preprocess, 10.2ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)


t:   0%|          | 2/1800 [00:00<02:27, 12.22it/s, now=None]


0: 384x640 3 cars, 1 truck, 10.5ms
Speed: 2.5ms preprocess, 10.5ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 truck, 15.4ms
Speed: 2.6ms preprocess, 15.4ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)


...


0: 384x640 3 cars, 2 trucks, 18.0ms
Speed: 2.2ms preprocess, 18.0ms inference, 4.1ms postprocess per image at shape (1, 3, 384, 640)


t:   6%|▋         | 114/1800 [00:27<09:33,  2.94it/s, now=None]


0: 384x640 4 cars, 1 truck, 20.9ms
Speed: 2.4ms preprocess, 20.9ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)


t:   6%|▋         | 115/1800 [00:27<09:33,  2.94it/s, now=None]


0: 384x640 4 cars, 2 trucks, 28.2ms
Speed: 2.1ms preprocess, 28.2ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)
